<a href="https://colab.research.google.com/github/olgagasowska/Machine-Learning-for-Linguists/blob/main/Vanilla-RNN_with_custom-built_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

import numpy as np
import torch
import torch.nn as nn


import torch
import torch.nn as nn
import torch.optim as optim


import numpy as np

# Base RNN Cell (Vanilla RNN)
class RNNCell:
    def __init__(self, input_size: int, hidden_size: int):
        """
        Initialize a basic RNN cell with Xavier-initialized weights.
        :param input_size: Number of input features.
        :param hidden_size: Number of units in the hidden layer.
        """
        self.hidden_size = hidden_size

        # Xavier initialization limits for Wx (input to hidden weights)
        fan_in_Wx = input_size
        fan_out_Wx = hidden_size
        limit_Wx = np.sqrt(6 / (fan_in_Wx + fan_out_Wx))

        # Xavier initialization limits for Wh (hidden to hidden weights)
        fan_in_Wh = hidden_size
        fan_out_Wh = hidden_size
        limit_Wh = np.sqrt(6 / (fan_in_Wh + fan_out_Wh))

        # Input to hidden weights and hidden to hidden weights using Xavier initialization
        self.Wx = np.random.uniform(-limit_Wx, limit_Wx, size=(input_size, hidden_size))  # Input to hidden weights
        self.Wh = np.random.uniform(-limit_Wh, limit_Wh, size=(hidden_size, hidden_size))  # Hidden to hidden weights

        # Bias for hidden layer initialized to zeros
        self.bh = np.zeros((1, hidden_size))

        # Tanh activation function
        self.tanh = np.tanh

    def forward(self, input_t: np.ndarray, h_prev: np.ndarray) -> np.ndarray:
        """
        Forward pass for a basic RNN cell.
        :param input_t: Input at time step t (batch_size x input_size).
        :param h_prev: Hidden state from the previous time step (batch_size x hidden_size).
        :return: Updated hidden state.
        """
        h_t = self.tanh(np.dot(input_t, self.Wx) + np.dot(h_prev, self.Wh) + self.bh)
        return h_t





class RecurrentLayer(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, cell_type: str = 'RNN', device='cpu'):
        """
        Initialize a recurrent layer with specified cell type (RNN, GRU, or LSTM).
        :param input_size: Number of input features.
        :param hidden_size: Number of units in the hidden layer.
        :param cell_type: Type of recurrent cell ('RNN', 'GRU', 'LSTM').
        """
        super(RecurrentLayer, self).__init__()
        self.hidden_size = hidden_size
        self.device = device

        # Initialize the appropriate cell type
        if cell_type == 'RNN':
            self.cell = nn.RNNCell(input_size, hidden_size, device=device)
        elif cell_type == 'GRU':
            self.cell = nn.GRUCell(input_size, hidden_size, device=device)
        elif cell_type == 'LSTM':
            self.cell = nn.LSTMCell(input_size, hidden_size, device=device)
        else:
            raise ValueError("Unsupported cell type. Choose from 'RNN', 'GRU', or 'LSTM'.")
        self.cell_type = cell_type

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the recurrent layer for a sequence of inputs.
        :param inputs: Input sequence (batch size x sequence length x input size).
        :return: Output sequence (batch size x sequence length x hidden size).
        """
        # Ensure inputs are on the correct device
        inputs = inputs.to(self.device)
        batch_size, seq_len, _ = inputs.shape

        # Initialize hidden and cell states
        h_t = torch.zeros(batch_size, self.hidden_size, device=self.device)
        c_t = torch.zeros(batch_size, self.hidden_size, device=self.device) if self.cell_type == 'LSTM' else None

        outputs = []

        for t in range(seq_len):
            input_t = inputs[:, t, :]  # Extract input at time t
            if self.cell_type == 'LSTM':
                h_t, c_t = self.cell(input_t, (h_t, c_t))
            else:
                h_t = self.cell(input_t, h_t)
            outputs.append(h_t)

        # Stack outputs to form the output tensor
        output_tensor = torch.stack(outputs, dim=1)
        return output_tensor





# Network that supports any recurrent layer
class SimpleRecurrentNetwork:
    def __init__(self, input_size: int, hidden_size: int, output_size: int, cell_type: str = 'RNN', learning_rate: float = 0.001):
        self.learning_rate = learning_rate
        self.recurrent_layer = RecurrentLayer(input_size, hidden_size, cell_type)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))
        self.sigmoid = lambda x: 1 / (1 + np.exp(-x))

    def forward(self, inputs: np.ndarray) -> np.ndarray:
        outputs = self.recurrent_layer.forward(inputs)
        final_outputs = []
        for t in range(outputs.shape[1]):
            h_t = outputs[:, t, :]
            output_t = self.sigmoid(np.dot(h_t, self.weights_hidden_output) + self.bias_output)
            final_outputs.append(output_t)
        return np.stack(final_outputs, axis=1)

    def compute_loss(self, predictions: np.ndarray, targets: np.ndarray) -> float:
        predictions_clipped = np.clip(predictions, 1e-10, 1 - 1e-10)
        loss = -np.mean(targets * np.log(predictions_clipped) + (1 - targets) * np.log(1 - predictions_clipped))
        return loss

    def train(self, inputs: np.ndarray, targets: np.ndarray, epochs: int = 1000):
        for epoch in range(epochs):
            predictions = self.forward(inputs)
            loss = self.compute_loss(predictions, targets)
            if epoch % 100 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, inputs: np.ndarray) -> np.ndarray:
        return self.forward(inputs)

# Example usage
inputs = np.random.randn(2, 5, 3)  # Example input (batch_size=2, sequence_length


class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, dropout):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden


class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch_size, hidden_dim]
        # encoder_outputs = [src_len, batch_size, hidden_dim]

        src_len = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        return torch.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, attention, dropout):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(hidden_dim + emb_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden, encoder_outputs):
        trg = trg.unsqueeze(0)
        embedded = self.dropout(self.embedding(trg))

        attn_weights = self.attention(hidden[-1], encoder_outputs)
        attn_weights = attn_weights.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        context = torch.bmm(attn_weights, encoder_outputs).permute(1, 0, 2)

        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.rnn(rnn_input, hidden)

        output = torch.cat((output.squeeze(0), context.squeeze(0)), dim=1)
        prediction = self.fc_out(output)

        return prediction, hidden, attn_weights


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)

        input = trg[0, :]
        for t in range(1, trg_len):
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs





import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
from datasets import load_dataset

# 1. Loading Dataset
dataset = load_dataset("stanfordnlp/imdb")

# 2. Tokenizing Dataset
model_id = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def process(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(process, batched=True)

# Convert the tokenized dataset to PyTorch tensors
def convert_to_tensors(dataset):
    input_ids = torch.tensor(dataset["input_ids"])
    labels = torch.tensor(dataset["label"])
    return TensorDataset(input_ids, labels)

# Prepare the training and test datasets
train_dataset = convert_to_tensors(tokenized_datasets['train'])
test_dataset = convert_to_tensors(tokenized_datasets['test'])

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)








class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.W1 = nn.Linear(hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch_size, hidden_size]
        # encoder_outputs: [batch_size, sequence_len, hidden_size]
        sequence_len = encoder_outputs.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, sequence_len, 1)

        energy = torch.tanh(self.W1(encoder_outputs) + self.W2(hidden))
        attention = self.v(energy).squeeze(2)  # [batch_size, sequence_len]
        attention_weights = torch.softmax(attention, dim=1)

        # Apply attention weights to encoder outputs to get context vector
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attention_weights

class SimpleRecurrentNetworkWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, cell_type='RNN', use_embedding=True, device='cpu'):
        super(SimpleRecurrentNetworkWithAttention, self).__init__()

        self.hidden_size = hidden_size
        self.device = device

        # Embedding layer if input is tokenized text
        self.use_embedding = use_embedding
        if self.use_embedding:
            self.embedding = nn.Embedding(input_size, hidden_size).to(device)

        # Recurrent Layer (RNN, GRU, or LSTM)
        self.recurrent_layer = RecurrentLayer(
            input_size if not use_embedding else hidden_size,
            hidden_size,
            cell_type,
            device=self.device
        )

        # Attention mechanism
        self.attention = Attention(hidden_size).to(device)

        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_size, output_size).to(device)

    def forward(self, inputs):
        # Ensure inputs are on the correct device
        inputs = inputs.to(self.device)

        # Apply embedding if enabled
        if self.use_embedding:
            embedded = self.embedding(inputs)
        else:
            embedded = inputs  # Use raw inputs directly if embedding is not used

        # Pass through the recurrent layer
        rnn_output = self.recurrent_layer(embedded)

        # Get the last hidden state
        h_t = rnn_output[:, -1, :]  # Last hidden state

        # Apply attention mechanism
        context, attention_weights = self.attention(h_t, rnn_output)

        # Final prediction
        output = self.fc(context)

        return output, attention_weights








        # 5. Initialize the model, criterion, and optimizer
vocab_size = tokenizer.vocab_size
hidden_size = 256
output_size = 2  # Binary classification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleRecurrentNetworkWithAttention(
    input_size=vocab_size,
    hidden_size=hidden_size,
    output_size=output_size,
    cell_type='RNN',
    device=device
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 6. Device setup
model.to(device)

# 7. Training loop
def train(model, dataloader, criterion, optimizer, scheduler, device, print_every=100):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs, _ = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()

        running_loss += loss.item()

        # Print every 100 batches
        if (batch_idx + 1) % 50 == 0:
            print(f"Batch [{batch_idx + 1}/{total_batches}], Loss: {loss.item():.4f}")

    # Step the learning rate scheduler after every epoch
    scheduler.step()

    epoch_loss = running_loss / total_batches
    print(f"Epoch completed. Average Loss: {epoch_loss:.4f}")

    return epoch_loss

# Helper function to decode input_ids back to words
def decode_input(inputs, tokenizer):
    # Decode the tensor of token IDs back into words (using the tokenizer)
    return tokenizer.decode(inputs, skip_special_tokens=True).split()

# 8. Evaluation loop with attention visualization
def evaluate(model, dataloader, criterion, device, tokenizer):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, attention_weights = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Visualizing the attention weights for the first batch in evaluation
            if total == labels.size(0):  # Print only for the first batch
                print(f"Input sentence (token IDs): {inputs[0].cpu().numpy()}")

                # Decode the input sentence
                words = decode_input(inputs[0], tokenizer)
                print(f"Decoded Input Sentence: {' '.join(words)}")

                # Attention weights for the first example
                attention_weights_np = attention_weights[0].cpu().numpy()

                # Print the words along with their attention weights
                print("Word attention distribution:")
                for word, attn in zip(words, attention_weights_np):
                    print(f"{word}: {attn:.4f}")

    accuracy = correct / total
    return total_loss / len(dataloader), accuracy


# 9. Training and evaluation
print(f"train loader size: {len(train_loader)}")
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, scheduler, device, print_every=100)

    # Evaluate the model and print attention visualization
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device, tokenizer)

    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print('-' * 50)

    import time

# Hyperparameters
N_EPOCHS = 5
CLIP = 1  # Gradient clipping value

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP, device)
    valid_loss = evaluate(model, valid_dataloader, criterion, device)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq_attention_model.pt')  # Save best model

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {torch.exp(torch.tensor(train_loss)):.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {torch.exp(torch.tensor(valid_loss)):.3f}')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

train loader size: 1563
Epoch 1/5
Batch [50/1563], Loss: 0.7076
Batch [100/1563], Loss: 0.6911
Batch [150/1563], Loss: 0.6982
Batch [200/1563], Loss: 0.6928
Batch [250/1563], Loss: 0.6905
Batch [300/1563], Loss: 0.6923
Batch [350/1563], Loss: 0.6925
Batch [400/1563], Loss: 0.6826
Batch [450/1563], Loss: 0.6896
Batch [500/1563], Loss: 0.6955
Batch [550/1563], Loss: 0.7024
Batch [600/1563], Loss: 0.6889
Batch [650/1563], Loss: 0.6950
Batch [700/1563], Loss: 0.6787
Batch [750/1563], Loss: 0.7022
Batch [800/1563], Loss: 0.6917
Batch [850/1563], Loss: 0.6822
Batch [900/1563], Loss: 0.6853
Batch [950/1563], Loss: 0.6907
Batch [1000/1563], Loss: 0.6912
Batch [1050/1563], Loss: 0.6964
Batch [1100/1563], Loss: 0.6852
Batch [1150/1563], Loss: 0.6915
Batch [1200/1563], Loss: 0.6972
Batch [1250/1563], Loss: 0.6756
Batch [1300/1563], Loss: 0.6932
Batch [1350/1563], Loss: 0.6696
Batch [1400/1563], Loss: 0.6860
Batch [1450/1563], Loss: 0.6822
Batch [1500/1563], Loss: 0.6862
Batch [1550/1563], Loss: 0

In [ ]:

print(model)

SimpleRecurrentNetworkWithAttention(
  (embedding): Embedding(250002, 256)
  (recurrent_layer): RecurrentLayer(
    (cell): RNNCell(256, 256)
  )
  (attention): Attention(
    (W1): Linear(in_features=256, out_features=256, bias=True)
    (W2): Linear(in_features=256, out_features=256, bias=True)
    (v): Linear(in_features=256, out_features=1, bias=False)
  )
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


In [ ]:
# Print the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

# Separate trainable and non-trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = total_params - trainable_params

print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {non_trainable_params}")



Total number of parameters: 64264450
Trainable parameters: 64264450
Non-trainable parameters: 0


In [ ]:
# Example list of sentences
sentences = [
    "i hate this movie",
    "this is the best film I've ever seen",
    "the plot was boring and slow",
    "i love the acting in this movie"
]

def classify(sentences, model, tokenizer, device):
    model.eval()  # Set model to evaluation mode

    # Iterate through each sentence in the list
    for sentence in sentences:
        print(f"\nProcessing sentence: '{sentence}'")

        # 1. Tokenize the input sentence
        inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

        # Move inputs to the correct device
        input_ids = inputs['input_ids'].to(device)

        # 2. Pass the tokenized sentence through the model to get predictions and attention weights
        with torch.no_grad():
            outputs, attention_weights = model(input_ids)

        # 3. Decode the input sentence to display the original words
        decoded_sentence = tokenizer.decode(input_ids[0], skip_special_tokens=True).split()

        # 4. Display the model's prediction
        _, predicted_label = torch.max(outputs, 1)
        predicted_label = predicted_label.item()
        sentiment = "Positive" if predicted_label == 1 else "Negative"
        print(f"Model Prediction: {sentiment}")

        # 5. Display the attention weights for each word in the input sentence
        attention_weights_np = attention_weights[0].cpu().numpy()  # Convert attention weights to NumPy array
        print("Word attention distribution:")
        for word, attn_weight in zip(decoded_sentence, attention_weights_np):
            print(f"{word}: {attn_weight:.4f}")

# Call the function with the list of sentences
classify(sentences, model, tokenizer, device)


Processing sentence: 'i hate this movie'
Model Prediction: Positive
Word attention distribution:
i: 0.0467
hate: 0.0579
this: 0.0380
movie: 0.0440

Processing sentence: 'this is the best film I've ever seen'
Model Prediction: Positive
Word attention distribution:
this: 0.0379
is: 0.0365
the: 0.0906
best: 0.0466
film: 0.0207
I've: 0.0057
ever: 0.0199
seen: 0.0095

Processing sentence: 'the plot was boring and slow'
Model Prediction: Negative
Word attention distribution:
the: 0.0166
plot: 0.0111
was: 0.0376
boring: 0.5671
and: 0.0266
slow: 0.0129

Processing sentence: 'i love the acting in this movie'
Model Prediction: Positive
Word attention distribution:
i: 0.0427
love: 0.0529
the: 0.0227
acting: 0.0158
in: 0.0348
this: 0.0128
movie: 0.0454
